<a href="https://colab.research.google.com/github/Jeongcc/data_analysis_practice/blob/main/lh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import KFold, GridSearchCV
import warnings
warnings.filterwarnings('ignore')

In [2]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso, LogisticRegression
# Ridge는 alpha Lasso도 alpha
# Logistic Regressio은 penalty와 regularization strength인 C값
from sklearn.svm import SVR
# epsilon 디폴트 0.1,  regularization C 1.0, gamma=auto 또는 'scale' , kernel = 'rbf', 'poly', 'sigmoid' 만 사용
from sklearn.neural_network import MLPRegressor
# hidden_layer_sizes = (100,) , (10, 10, ) 정도만 activation = 'relu', 'logistic'까지만, alpha =0.0001, solver = 'lbfgs', 'adam'까지만

In [6]:
from google.colab import files
myfile = files.upload()

Saving train.csv to train.csv


In [7]:
dt = pd.read_csv("train.csv")[["일자", "요일", "본사정원수", "본사휴가자수", "본사출장자수", "본사시간외근무명령서승인건수", "현본사소속재택근무자수", "중식계", "석식계"]]

prior = pd.DataFrame({"prior": [None]})
posterior = pd.DataFrame({"posterior": []})
prior = prior.append(pd.DataFrame({"prior": list(dt["요일"][0:len(dt["요일"])-1])}), ignore_index=True)
posterior = posterior.append(pd.DataFrame({"posterior": list(dt["요일"][1:len(dt["요일"])])}), ignore_index=True)
posterior = posterior.append(pd.DataFrame({"posterior": [None]}), ignore_index=True)

dt = dt.join(prior)
dt = dt.join(posterior)
# prior의 0행과 posterior의 끝 행에 null 값을 추가하기

In [8]:
dt.loc[(dt["요일"]=="월") & (dt["posterior"]=="화"), "posterior"] = 1
dt.loc[(dt["요일"]=="화") & (dt["posterior"]=="수"), "posterior"] = 1
dt.loc[(dt["요일"]=="수") & (dt["posterior"]=="목"), "posterior"] = 1
dt.loc[(dt["요일"]=="목") & (dt["posterior"]=="금"), "posterior"] = 1
dt.loc[(dt["요일"]=="금") & (dt["posterior"]=="월"), "posterior"] = 1

dt.loc[(dt["요일"]=="월") & (dt["prior"]=="금"), "prior"] = 1
dt.loc[(dt["요일"]=="화") & (dt["prior"]=="월"), "prior"] = 1
dt.loc[(dt["요일"]=="수") & (dt["prior"]=="화"), "prior"] = 1
dt.loc[(dt["요일"]=="목") & (dt["prior"]=="수"), "prior"] = 1
dt.loc[(dt["요일"]=="금") & (dt["prior"]=="목"), "prior"] = 1

# 0으로 표현된 데이터는 그 다음 요일이나 그전 요일을 모르거나 휴일이 껴있을 때
dt.loc[dt["posterior"]!=1, "posterior"] = 0
dt.loc[dt["prior"]!=1, "prior"] = 0

dt

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,중식계,석식계,prior,posterior
0,2016-02-01,월,2601,50,150,238,0.0,1039.0,331.0,0,1
1,2016-02-02,화,2601,50,173,319,0.0,867.0,560.0,1,1
2,2016-02-03,수,2601,56,180,111,0.0,1017.0,573.0,1,1
3,2016-02-04,목,2601,104,220,355,0.0,978.0,525.0,1,1
4,2016-02-05,금,2601,278,181,34,0.0,925.0,330.0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
1200,2021-01-20,수,2983,75,198,4,391.0,1093.0,421.0,1,1
1201,2021-01-21,목,2983,92,231,462,351.0,832.0,353.0,1,1
1202,2021-01-22,금,2983,255,248,1,303.0,579.0,217.0,1,1
1203,2021-01-25,월,2983,107,153,616,327.0,1145.0,502.0,1,1


In [9]:
# 원핫인코딩
dt = pd.get_dummies(dt, columns=['요일'])

In [10]:
dt

,일자,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,중식계,석식계,prior,posterior,요일_금,요일_목,요일_수,요일_월,요일_화
0,2016-02-01,2601,50,150,238,0.0,1039.0,331.0,0,1,0,0,0,1,0
1,2016-02-02,2601,50,173,319,0.0,867.0,560.0,1,1,0,0,0,0,1
2,2016-02-03,2601,56,180,111,0.0,1017.0,573.0,1,1,0,0,1,0,0
3,2016-02-04,2601,104,220,355,0.0,978.0,525.0,1,1,0,1,0,0,0
4,2016-02-05,2601,278,181,34,0.0,925.0,330.0,1,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1200,2021-01-20,2983,75,198,4,391.0,1093.0,421.0,1,1,0,0,1,0,0
1201,2021-01-21,2983,92,231,462,351.0,832.0,353.0,1,1,0,1,0,0,0
1202,2021-01-22,2983,255,248,1,303.0,579.0,217.0,1,1,1,0,0,0,0
1203,2021-01-25,2983,107,153,616,327.0,1145.0,502.0,1,1,0,0,0,1,0


In [11]:
X1_train, X1_test, y1_train, y1_test = train_test_split(dt.iloc[:,range(1,len(dt.columns)-1)], dt[["중식계"]])
X2_train, X2_test, y2_train, y2_test = train_test_split(dt.iloc[:,range(1,len(dt.columns)-1)], dt[["석식계"]])

In [12]:
pipe = Pipeline([('preprocessing', None), ('regressor', LinearRegression())])
pre_list = [StandardScaler(), MinMaxScaler(), None]
hyperparam_grid = [
    {"regressor": [LinearRegression()], 'preprocessing': pre_list},
    {"regressor": [Ridge()], 'preprocessing': pre_list,
    "regressor__alpha": [0.0001, 0.001, 0.01, 0.1, 1, 10]},
    {"regressor": [Lasso()], 'preprocessing': pre_list,
    "regressor__alpha": [0.0001, 0.001, 0.01, 0.1, 1, 10]},
    {"regressor": [LogisticRegression()], 'preprocessing': pre_list,
    "regressor__C": [0.0001, 0.001, 0.01, 0.1, 1, 10]},
    {"regressor": [SVR()], 'preprocessing': pre_list,
    "regressor__epsilon": [0.001, 0.01, 0.1, 1, 10],
    "regressor__C": [0.0001, 0.001, 0.01, 0.1, 1, 10]},
    {"regressor": [MLPRegressor()], 'preprocessing': pre_list,
    "regressor__hidden_layer_sizes": [(100,) , (10, 10, )],
    "regressor__activation": ["relu", "logistic"],
    "regressor__solver": ["lbfgs", "adam"],
    "regressor__alpha": [0.0001, 0.01, 1]}

]
kfold = KFold(n_splits=5, shuffle=True, random_state=1)

grid1 = GridSearchCV(pipe, hyperparam_grid, scoring="neg_mean_absolute_error",
                   refit=True, cv=kfold)
grid2 = GridSearchCV(pipe, hyperparam_grid, scoring="neg_mean_absolute_error",
                   refit=True, cv=kfold)

In [13]:
grid1.fit(X1_train, y1_train)
print(grid1.best_estimator_)
print(grid1.best_params_)
print(grid1.best_score_)
print(grid1.score(X1_test, y1_test))

Pipeline(memory=None,
         steps=[('preprocessing', None),
                ('regressor',
                 LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
                                  normalize=False))],
         verbose=False)
{'preprocessing': None, 'regressor': LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)}
-1.593552382336372e-13
-1.976344033240014e-13


In [14]:
grid2.fit(X2_train, y2_train)
print(grid2.best_estimator_)
print(grid2.best_params_)
print(grid2.best_score_)
print(grid2.score(X2_test, y2_test))

Pipeline(memory=None,
         steps=[('preprocessing', None),
                ('regressor',
                 LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
                                  normalize=False))],
         verbose=False)
{'preprocessing': None, 'regressor': LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)}
-1.0473401159594413e-13
-7.898271582991497e-14


In [16]:
from sklearn.metrics import mean_absolute_error
print("중식 최종 결과: ", mean_absolute_error(y1_test, grid1.best_estimator_.predict(X1_test)))
print("석식 최종 결과: ", mean_absolute_error(y2_test, grid2.best_estimator_.predict(X2_test)))

중식 최종 결과:  1.976344033240014e-13
석식 최종 결과:  7.898271582991497e-14
